<a href="https://colab.research.google.com/github/santoshrsarangi/tensorflow/blob/main/NLP/RNN_IMDB_Movie_Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  21.7M      0  0:00:03  0:00:03 --:--:-- 21.7M


In [34]:
!tar -xf aclImdb_v1.tar.gz

In [35]:
!ls -l

total 82164
drwxr-xr-x 4 7297 1000     4096 Jun 26  2011 aclImdb
-rw-r--r-- 1 root root 84125825 Dec 26 02:25 aclImdb_v1.tar.gz
drwxr-xr-x 1 root root     4096 Dec 20 20:19 sample_data


In [39]:
!rm -r aclImdb/train/unsup

In [40]:
!cat aclImdb/train/pos/4077_10.txt

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy

In [41]:
import os, pathlib, shutil, random

In [42]:
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"

In [50]:
for category in ("pos", "neg"):
  os.makedirs(val_dir / category)
  files = os.listdir(train_dir / category)
  random.Random(1337).shuffle(files)
  num_of_val_files = int(len(files) * 0.2)

  val_files = files[-num_of_val_files:]

  for fname in val_files:
    shutil.move(train_dir / category / fname, val_dir / category / fname)

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers


In [2]:
from keras.layers import TextVectorization

In [3]:
text_vectorization = TextVectorization(output_mode="int")

In [4]:
import re
import string

In [5]:
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [6]:
from numpy import string_
def custom_standardization_fn(string_tensor):
  lowercase_string = tf.strings.lower(string_tensor)
    

In [25]:
dataset = [
    "hello, this, is a sample text which I'd like to test on Text Vecrotization leayer of keras. This should/, help in NLP"
]

In [26]:
text_vectorization.adapt(dataset)

In [27]:
vocab=text_vectorization.get_vocabulary()

In [28]:
encoded_sentense = text_vectorization("Hello, how far have you learned NLP Text Vectorization using Keras")

In [29]:
print(encoded_sentense)

tf.Tensor([20  1  1  1  1  1 12  3  1  1 15], shape=(11,), dtype=int64)


In [30]:
inverse_vocab = dict(enumerate(vocab))

In [31]:
decoded_sentense = " ".join(inverse_vocab[int(i)] for i in encoded_sentense)

In [32]:
print(decoded_sentense)

hello [UNK] [UNK] [UNK] [UNK] [UNK] nlp text [UNK] [UNK] keras
